In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE208668"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE208668"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE208668.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE208668.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE208668.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sleep Disturbance and Activation of Cellular and Transcriptional Mechanisms of Inflammation in Older Adults"
!Series_summary	"Genome-wide transcriptional profiling results were used to systematically assess the extent to which transcriptomes of older adults with insomnia show expression of genes that are different from those without insomnia"
!Series_overall_design	"Total RNA obtained from peripheral blood mononuclear cells (PBMCs) of older adults with insomnia disorder who participated in the Behavioral Treatment of Insomnia in Aging study (n = 17) and older adults without insomnia disorder who participated in the Sleep Health and Aging Research (SHARE) study (n = 25) at UCLA."
!Series_overall_design	""
!Series_overall_design	"**Please note that raw data was lost and thus is not included in the records**"
Sample Characteristics Dictionary:
{0: ['insomnia: yes', 'insomnia: no'], 1: ['age: 65', 'age: 75', 'age: 77', 'age: 64', 'age: 60', 'age: 67',

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The background information mentions "genome-wide transcriptional profiling" and specifically mentions "Total RNA" 
# from PBMCs - this indicates gene expression data. The note about raw data being lost is concerning,
# but since we're working with processed matrix data, we'll proceed cautiously.
is_gene_available = True  # The dataset should contain gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For the trait (Depression), we need to look at relevant variables in this insomnia study
# From sample characteristics, row 9 contains "history of depression" which is relevant for our trait
trait_row = 9

# Age is available in row 1
age_row = 1

# Gender is available in row 2
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert depression history data to binary format (0 = no, 1 = yes)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value == 'yes':
        return 1
    elif value == 'no':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to continuous format"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
    else:
        age_str = value.strip()
    
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 = female, 1 = male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
    else:
        gender = value.strip().lower()
    
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the sample characteristics from the provided dictionary in the previous output
    # This assumes that the sample characteristics data is accessible from a matrix file
    # We need to load the actual matrix file here
    try:
        matrix_file = f"{in_cohort_dir}/matrix.csv"
        clinical_data = pd.read_csv(matrix_file, skiprows=0)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except FileNotFoundError:
        print(f"Could not find matrix file at {in_cohort_dir}/matrix.csv")
        print("Clinical data extraction is unavailable for this cohort.")


Could not find matrix file at ../../input/GEO/Depression/GSE208668/matrix.csv
Clinical data extraction is unavailable for this cohort.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE208668/GSE208668_series_matrix.txt.gz
Gene data shape: (33210, 42)
First 20 gene/probe identifiers:
Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be a mix of human gene symbols and potentially some probe identifiers
# Examples like "A1BG", "A2M", "AACS" are valid human gene symbols
# However, some like "7A5" and "AAA1" may be probe identifiers or alternative names
# Since the majority appear to be gene symbols already, we won't need extensive mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails
    
# 2. Recreate clinical data using correct row indices from step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using correct row indices and conversion functions
def convert_trait(value):
    """Convert depression history data to binary format (0 = no, 1 = yes)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value == 'yes':
        return 1
    elif value == 'no':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to continuous format"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
    else:
        age_str = value.strip()
    
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 = female, 1 = male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
    else:
        gender = value.strip().lower()
    
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# Use correct row indices identified in step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,  # Using the trait variable from context (Depression)
    trait_row=9,  # Using row 9 for depression history as identified in step 2
    convert_trait=convert_trait,
    age_row=1,    # Age data is in row 1
    convert_age=convert_age,
    gender_row=2, # Gender data is in row 2
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood mononuclear cells of older adults with and without depression history, from a study on insomnia disorder."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (19539, 42)


Normalized gene data saved to ../../output/preprocess/Depression/gene_data/GSE208668.csv
Selected clinical data shape: (3, 42)
Clinical data preview:
{'GSM6360934': [1.0, 65.0, 0.0], 'GSM6360935': [0.0, 75.0, 1.0], 'GSM6360936': [1.0, 77.0, 0.0], 'GSM6360937': [0.0, 64.0, 0.0], 'GSM6360938': [1.0, 60.0, 1.0], 'GSM6360939': [1.0, 67.0, 0.0], 'GSM6360940': [1.0, 72.0, 1.0], 'GSM6360941': [0.0, 62.0, 1.0], 'GSM6360942': [0.0, 73.0, 0.0], 'GSM6360943': [0.0, 74.0, 1.0], 'GSM6360944': [0.0, 73.0, 1.0], 'GSM6360945': [0.0, 68.0, 0.0], 'GSM6360946': [0.0, 62.0, 0.0], 'GSM6360947': [1.0, 73.0, 0.0], 'GSM6360948': [0.0, 70.0, 0.0], 'GSM6360949': [0.0, 60.0, 0.0], 'GSM6360950': [1.0, 61.0, 0.0], 'GSM6360951': [0.0, 66.0, 0.0], 'GSM6360952': [0.0, 69.0, 0.0], 'GSM6360953': [0.0, 62.0, 1.0], 'GSM6360954': [1.0, 67.0, 0.0], 'GSM6360955': [1.0, 62.0, 0.0], 'GSM6360956': [0.0, 71.0, 1.0], 'GSM6360957': [0.0, 63.0, 1.0], 'GSM6360958': [1.0, 62.0, 1.0], 'GSM6360959': [0.0, 61.0, 0.0], 'GSM6360960': [1.

Data shape after handling missing values: (42, 19541)
For the feature 'Depression', the least common label is '1.0' with 17 occurrences. This represents 40.48% of the dataset.
The distribution of the feature 'Depression' in this dataset is fine.

Quartiles for 'Age':
  25%: 63.0
  50% (Median): 68.5
  75%: 73.0
Min: 60.0
Max: 80.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 18 occurrences. This represents 42.86% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Depression/GSE208668.csv
